*We use Python 3.8 for all labs in this subject.*

Train BPE on a toy text example

bpe algorithm: https://web.stanford.edu/~jurafsky/slp3/2.pdf (2.4.3)

In [1]:
import re, collections
from pprint import pprint

text = "The aims for this subject is for students to develop an understanding of the main algorithms used in natural language processing, for use in a diverse range of applications including text classification, machine translation, and question answering. Topics to be covered include part-of-speech tagging, n-gram language modelling, syntactic parsing and deep learning. The programming language used is Python, see for more information on its use in the workshops, assignments and installation at home."
# text = 'low '*5 +'lower '*2+'newest '*6 +'widest '*3
def get_vocab(text):
    vocab = collections.defaultdict(int)
    for word in text.strip().split():
        #note: we use the special token </w> (instead of underscore in the lecture) to denote the end of a word
        vocab[' '.join(list(word)) + ' </w>'] += 1
    return vocab

In [2]:
def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens

In [3]:
vocab = get_vocab(text)
print("Vocab =", vocab)
print('==========')
print('Tokens Before BPE')
tokens = get_tokens(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))

Vocab = defaultdict(<class 'int'>, {'T h e </w>': 2, 'a i m s </w>': 1, 'f o r </w>': 4, 't h i s </w>': 1, 's u b j e c t </w>': 1, 'i s </w>': 2, 's t u d e n t s </w>': 1, 't o </w>': 2, 'd e v e l o p </w>': 1, 'a n </w>': 1, 'u n d e r s t a n d i n g </w>': 1, 'o f </w>': 2, 't h e </w>': 2, 'm a i n </w>': 1, 'a l g o r i t h m s </w>': 1, 'u s e d </w>': 2, 'i n </w>': 3, 'n a t u r a l </w>': 1, 'l a n g u a g e </w>': 3, 'p r o c e s s i n g , </w>': 1, 'u s e </w>': 2, 'a </w>': 1, 'd i v e r s e </w>': 1, 'r a n g e </w>': 1, 'a p p l i c a t i o n s </w>': 1, 'i n c l u d i n g </w>': 1, 't e x t </w>': 1, 'c l a s s i f i c a t i o n , </w>': 1, 'm a c h i n e </w>': 1, 't r a n s l a t i o n , </w>': 1, 'a n d </w>': 3, 'q u e s t i o n </w>': 1, 'a n s w e r i n g . </w>': 1, 'T o p i c s </w>': 1, 'b e </w>': 1, 'c o v e r e d </w>': 1, 'i n c l u d e </w>': 1, 'p a r t - o f - s p e e c h </w>': 1, 't a g g i n g , </w>': 1, 'n - g r a m </w>': 1, 'm o d e l l i n g ,

In [4]:
def get_stats(vocab):
    """
    This function iterates through all words in the vocabulary and count pair of tokens which are next to each other.
    
    EXAMPLE:
        word = 'T h e <\w>'
        pairs of tokens in this word [('T', 'h'), ('h', 'e'), ('e', '<\w>')]
        
    INPUT:
        vocab: Dict[str, int]  # The vocabulary, a counter for word frequency
        
    OUTPUT:
        pairs: Dict[Tuple[str, str], int] # Word pairs, a counter for pair frequency
    
    """
    pairs = collections.defaultdict(int)
    ###
    # Your answer BEGINS HERE
    ###
    for key in vocab.keys():
        token_list = key.split(' ')  # tokenise by space
        for i in range(len(token_list) - 1):
            pairs[(token_list[i], token_list[i+1])] += 1
    ###
    # Your answer ENDS HERE
    ###
    return pairs

In [5]:
def merge_vocab(pair, v_in):
    """
    This function merges a given pair of tokens in all words in the vocabulary
    
    EXAMPLE:
        word = 'T h e <\w>'
        pair = ('e', '<\w>')
        word_after_merge = 'T h e<\w>'
        
    Input:
        pair: Tuple[str, str] # the pair of tokens need to be merged
        v_in: Dict[str, int]  # vocabulary before merge
        
    Output:
        v_out: Dict[str, int] # vocabulary after merge
        
    HINT:
        When merging pair ('h', 'e') for word 'Th e<\w>', the two tokens in this word 'Th' and 'e<\w>' shouldn't be merged.
    
    """
    v_out = collections.defaultdict(int)
    ###
    # Your answer BEGINS HERE
    ###
    for word in v_in:
        v_out[word.replace(' '.join(pair), ''.join(pair))] += 1
    ###
    # Your answer ENDS HERE
    ###
    return v_out

In [6]:
#about 100 merges we start to see common words
num_merges = 100
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    new_token = ''.join(best)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    # add new token to the vocab
    tokens[new_token] = pairs[best]
    # deduct frequency for tokens have been merged
    tokens[best[0]] -= pairs[best]
    tokens[best[1]] -= pairs[best]
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)))
    print('==========')

Iter: 0
Best pair: ('i', 'n')
Tokens: defaultdict(<class 'int'>, {'T': 3, 'h': 11, 'e': 39, '</w>': 73, 'a': 38, 'i': 21, 'm': 12, 's': 34, 'f': 9, 'o': 29, 'r': 22, 't': 29, 'u': 14, 'b': 2, 'j': 1, 'c': 13, 'd': 15, 'n': 29, 'v': 3, 'l': 16, 'p': 11, 'g': 22, ',': 7, 'x': 1, 'q': 1, 'w': 2, '.': 3, '-': 3, 'y': 2, 'P': 1, 'k': 1, 'in': 16})
Number of tokens: 32
Iter: 1
Best pair: ('e', '</w>')
Tokens: defaultdict(<class 'int'>, {'T': 3, 'h': 11, 'e': 28, '</w>': 62, 'a': 38, 'i': 21, 'm': 12, 's': 34, 'f': 9, 'o': 29, 'r': 22, 't': 29, 'u': 14, 'b': 2, 'j': 1, 'c': 13, 'd': 15, 'n': 29, 'v': 3, 'l': 16, 'p': 11, 'g': 22, ',': 7, 'x': 1, 'q': 1, 'w': 2, '.': 3, '-': 3, 'y': 2, 'P': 1, 'k': 1, 'in': 16, 'e</w>': 11})
Number of tokens: 33
Iter: 2
Best pair: ('s', '</w>')
Tokens: defaultdict(<class 'int'>, {'T': 3, 'h': 11, 'e': 28, '</w>': 53, 'a': 38, 'i': 21, 'm': 12, 's': 25, 'f': 9, 'o': 29, 'r': 22, 't': 29, 'u': 14, 'b': 2, 'j': 1, 'c': 13, 'd': 15, 'n': 29, 'v': 3, 'l': 16, 'p': 

After training, used the BPE dictionaries to tokenise sentences

In [7]:
def get_vocab_tokenization(vocab):
    vocab_tokenization = {}
    for word, freq in vocab.items():
        word_tokens = word.split()
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return vocab_tokenization

In [8]:
#display the vocab
vocab_tokenization = get_vocab_tokenization(vocab)
vocab_tokenization

{'The</w>': ['The</w>'],
 'aims</w>': ['aims</w>'],
 'for</w>': ['for</w>'],
 'this</w>': ['this</w>'],
 'subject</w>': ['subject</w>'],
 'is</w>': ['is</w>'],
 'students</w>': ['students</w>'],
 'to</w>': ['to</w>'],
 'develop</w>': ['develop</w>'],
 'an</w>': ['an</w>'],
 'understanding</w>': ['understanding</w>'],
 'of</w>': ['of</w>'],
 'the</w>': ['the</w>'],
 'main</w>': ['main</w>'],
 'algorithms</w>': ['algorithms</w>'],
 'used</w>': ['used</w>'],
 'in</w>': ['in</w>'],
 'natural</w>': ['natural</w>'],
 'language</w>': ['language</w>'],
 'processing,</w>': ['pro', 'c', 'e', 'ss', 'ing,</w>'],
 'use</w>': ['us', 'e</w>'],
 'a</w>': ['a', '</w>'],
 'diverse</w>': ['d', 'i', 'ver', 's', 'e</w>'],
 'range</w>': ['r', 'ang', 'e</w>'],
 'applications</w>': ['a', 'p', 'p', 'l', 'ication', 's</w>'],
 'including</w>': ['inclu', 'ding</w>'],
 'text</w>': ['t', 'e', 'x', 't</w>'],
 'classification,</w>': ['cl', 'assi', 'f', 'ication', ',</w>'],
 'machine</w>': ['ma', 'c', 'h', 'in', 'e</w

In [9]:
def measure_token_length(token):
    if token[-4:] == '</w>':
        return len(token[:-4]) + 1
    else:
        return len(token)

In [10]:
def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    if string == '':
        return []
    if sorted_tokens == []:
        return [unknown_token] * len(string)

    string_tokens = []
    # iterate over all tokens to find match
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        token_reg = re.escape(token)
        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        # if no match found in the string, go to next token
        if len(matched_positions) == 0:
            continue
        # collect end position of matches in the string
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]
        substring_start_position = 0
        for substring_end_position in substring_end_positions:
            # slice for sub-word
            substring = string[substring_start_position:substring_end_position]
            # tokenize this sub-word with tokens remaining
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        # tokenize the remaining string
        remaining_substring = string[substring_start_position:]
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    else:
        # return list of unknown token if no match is found for the string
        string_tokens = [unknown_token] * len(string)
        
    return string_tokens

In [11]:
def sort_tokens(tokens_frequencies):
    """
    This function generates a list of all tokens sorted by their length (1st key) and frequency (2nd key).
    
    EXAMPLE:
        token frequency dictionary before sorting: {'natural': 3, 'language':2, 'processing': 4, 'lecture': 4}
        sorted tokens: ['processing', 'language', 'lecture', 'natural']
        
    INPUT:
        token_frequencies: Dict[str, int] # Counter for token frequency
        
    OUTPUT:
        sorted_token: List[str] # Tokens sorted by length and frequency
    
    """
    ###
    # Your answer BEGINS HERE
    ###
    sorted_tokens = dict(sorted(tokens_frequencies.items(),
                               key=lambda word_freq: word_freq[1],
                               reverse=True))
    ###
    # Your answer ENDS HERE
    ###
    return list(sorted_tokens.keys())

In [12]:
#sort tokens by length and frequency
sorted_tokens = sort_tokens(tokens)
print("Tokens =", sorted_tokens, "\n")

Tokens = ['</w>', 'e', 's', 'a', 'n', 'o', 't', 'e</w>', 'h', 'r', 'l', 'g', 'i', 'f', 'u', 'd', 'm', 'c', 'p', ',</w>', '-', 'in', 's</w>', 'ing,</w>', 'T', 'w', 'y', 'on', 'or', 'de', 'st', 'ing</w>', 'ic', 'op', 'ma', 'in</w>', 'pro', 'ver', 'ication', 'inclu', 'assi', 'ans', 'lation', 'ing.</w>', 'par', 'gram', 'me', 'b', 'x', 'q', 'P', 'k', 'an', 'at', 'ion', 'ation', 'th', 'nt', 'al', 'd</w>', 'ss', 'cl', '.</w>', 'ar', 'for', 'is</w>', 'ec', 't</w>', 'nts</w>', 'ding</w>', 'of', 'us', 'ed</w>', 'ang', 'ag', 'The</w>', 'aims</w>', 'for</w>', 'this</w>', 'subject</w>', 'students</w>', 'to</w>', 'develop</w>', 'an</w>', 'understanding</w>', 'of</w>', 'the</w>', 'algorithms</w>', 'used</w>', 'natural</w>', 'language</w>', 'j', 'v', ',', '.', 'ing', 've', 'ms</w>', 'pr', 'incl', 'ass', 'gr', 'gra', 'Th', 'ai', 'su', 'sub', 'subj', 'subjec', 'stu', 'stude', 'to', 'deve', 'devel', 'develop', 'un', 'unde', 'under', 'underst', 'understan', 'alg', 'algor', 'algori', 'algorith', 'nat', 'na

In [13]:
sentence_1 = 'I like natural language processing!'
sentence_2 = 'I like natural languaaage processing!'
sentence_list = [sentence_1, sentence_2]

for sentence in sentence_list:
    
    print('==========')
    print("Sentence =", sentence)
    
    for word in sentence.split():
        word = word + "</w>"

        print('Tokenizing word: {}...'.format(word))
        if word in vocab_tokenization:
            print(vocab_tokenization[word])
        else:
            print(tokenize_word(string=word, sorted_tokens=sorted_tokens, unknown_token='</u>'))


Sentence = I like natural language processing!
Tokenizing word: I</w>...
['</u>', '</w>']
Tokenizing word: like</w>...
['l', 'i', 'k', 'e', '</w>']
Tokenizing word: natural</w>...
['natural</w>']
Tokenizing word: language</w>...
['language</w>']
Tokenizing word: processing!</w>...
['p', 'r', 'o', 'c', 'e', 's', 's', 'i', 'n', 'g', '</u>', '</w>']
Sentence = I like natural languaaage processing!
Tokenizing word: I</w>...
['</u>', '</w>']
Tokenizing word: like</w>...
['l', 'i', 'k', 'e', '</w>']
Tokenizing word: natural</w>...
['natural</w>']
Tokenizing word: languaaage</w>...
['l', 'a', 'n', 'g', 'u', 'a', 'a', 'a', 'g', 'e', '</w>']
Tokenizing word: processing!</w>...
['p', 'r', 'o', 'c', 'e', 's', 's', 'i', 'n', 'g', '</u>', '</w>']
